In [57]:
import pandas as pd
import os
import torchmetrics as tm
import torch

* Model de base à 0.91 (recall 0.17) : `roberta-bce_2022-03-25T23-01-17-833309`
* Model en enlevant digit et ponc (pas mieux) : `roberta-bce_2022-03-26T17-37-48-467028`
* 

In [58]:
# Dataset
DATA_DIR_PATH = os.path.abspath("../../data")
SESSION_DIR_PATH = os.path.abspath("../../session")
TRAIN_DATASET_PATH = os.path.join(DATA_DIR_PATH, "jigsaw2019-train.csv")
TEST_DATASET_PATH = os.path.join(DATA_DIR_PATH, "jigsaw2019-test.csv")
LABEL_LIST = ['toxicity', 'obscene', 'sexual_explicit',
            'identity_attack', 'insult', 'threat']
IDENTITY_LIST = ['male', 'female', 'transgender', 'other_gender', 'heterosexual',
                'homosexual_gay_or_lesbian', 'bisexual','other_sexual_orientation',
                'christian', 'jewish', 'muslim', 'hindu','buddhist', 'atheist',
                'other_religion', 'black', 'white', 'asian', 'latino',
                'other_race_or_ethnicity', 'physical_disability',
                'intellectual_or_learning_disability',
                'psychiatric_or_mental_illness','other_disability']
SELECTED_IDENTITY_LIST = ['male', 'female', 'black', 'white', 'homosexual_gay_or_lesbian',
                    'christian', 'jewish', 'muslim', 'psychiatric_or_mental_illness']


In [59]:
SELECTED_SESSION_NAME = "roberta-bce_2022-03-26T17-37-48-467028"
SELECTED_SESSION_PATH = os.path.join(SESSION_DIR_PATH, SELECTED_SESSION_NAME)
SESSION_TEST_FILE_PATH = os.path.join(SELECTED_SESSION_PATH, f"{SELECTED_SESSION_NAME}.test.csv")

In [60]:
pred_test_df = pd.read_csv(SESSION_TEST_FILE_PATH, index_col=0)
target_test_df = pd.read_csv(TEST_DATASET_PATH, index_col=0)

In [61]:
train_df = pd.read_csv(TRAIN_DATASET_PATH, index_col=0)

In [63]:
(target_test_df.toxicity>=0.5).shape

(194292,)

In [64]:
target_test_df[LABEL_LIST] = (target_test_df[LABEL_LIST] >= 0.5).astype(int)
target_test_df = target_test_df[~target_test_df.white.isna()]

In [65]:
accuracy = tm.Accuracy()
f1score = tm.F1Score()
recall = tm.Recall()
precision = tm.Precision()
auroc = tm.AUROC()

/work2/home/ing1/corentin/hatespeech-detection-models/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [66]:
print(pred_test_df.toxicity.shape)
print(target_test_df.toxicity.shape)

(42823,)
(42823,)


In [67]:
pred = torch.Tensor(pred_test_df.toxicity.values)
target = torch.Tensor(target_test_df.toxicity.values).to(int)

In [68]:
accuracy(pred,target)

tensor(0.8994)

In [69]:
pred_i= torch.Tensor((pred_test_df.toxicity>=0.5).astype(int))
f1score(pred_i,target)

tensor(0.2958)

In [70]:
precision(pred, target)

tensor(0.6650)

In [71]:
recall(pred, target)

tensor(0.1902)

In [72]:
auroc(pred,target)

tensor(0.8592)

In [73]:
pred_multi= torch.Tensor(pred_test_df[LABEL_LIST].to_numpy())
target_multi= torch.Tensor((target_test_df[LABEL_LIST]>=0.5).astype(int).values).to(int)

In [74]:
f1score_multi = tm.F1Score()
f1score_multi(pred_multi, target_multi)

tensor(0.2438)